# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

### LangChain Tools


In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name

'search'

In [5]:
search.description

'search(query: str) -> str - Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

###### Add description to the function, to let the model understand the goal of it, by creating a pydantic class

In [7]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


###### Functions must have description when we use tool decorator

In [8]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [9]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [10]:
search.run("sf")

'42f'

#### 1. Define a function that take latitude&longitude of a place and return its current temperature, by calling an Api 


In [11]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

# Define the function that calls the Api
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        # print(results)
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    # print("current_utc_time", current_utc_time)
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    # print("\n time_list",time_list)
    temperature_list = results['hourly']['temperature_2m']
    # print("\n temperature_list",temperature_list)
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [12]:
get_current_temperature.name

'get_current_temperature'

In [13]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [14]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [15]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 36.2°C'

In [76]:
get_current_temperature.run({"latitude": 13, "longitude": 14})

'The current temperature is 35.5°C'

##### Convert the function tool into an openAi function

In [16]:
from langchain.tools.render import format_tool_to_openai_function

In [17]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

#### 2. Define a function that takes a query and search for it in the wikipedia, then return the first three pages and thier titles

In [24]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    # print("page titles", page_titles)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            # print("\n wikipage", wiki_page)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [26]:
search_wikipedia.name

'search_wikipedia'

In [27]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [28]:
search_wikipedia({"query": "donald trump"})

'Page: Donald Trump\nSummary: Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who has been the 47th and current president of the United States since 2025. A member of the Republican Party, he previously served as the 45th president from 2017 to 2021.\nTrump graduated with a bachelor\'s degree in economics from the University of Pennsylvania in 1968. He became president of the Trump family real estate business in 1971 and focused on luxury accommodations. After a series of business bankruptcies in the 1990s, he launched several side ventures. From 2004 to 2015, he hosted and co-produced the reality television series The Apprentice.\nTrump won the 2016 presidential election. His immigration policy included a travel ban targeting Muslims and refugees and expanding the U.S.–Mexico border wall; he also briefly implemented a family separation policy. He rolled back more than 100 environmental policies and regulations, signed the Tax Cuts a

###### convert the function into openapi function to be used later on 

In [29]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

### Create a list of openApi functions from the spec of an Api
    Where this step make it easy to communicate with Api that require specific inputs &outputs

In [30]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [31]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [32]:
# transform the api spec into openApi spec
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [41]:
#spec

In [34]:
# extract the functions from the openApi spec
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [35]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [36]:
pet_callables

<function langchain.chains.openai_functions.openapi.openapi_spec_to_openai_fn.<locals>.default_call_api(name: 'str', fn_args: 'dict', headers: 'Optional[dict]' = None, params: 'Optional[dict]' = None, **kwargs: 'Any') -> 'Any'>

In [37]:
from langchain.chat_models import ChatOpenAI

In [38]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [39]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{"params":{"limit":3}}'}})

In [40]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{"path_params":{"petId":"42"}}'}})

In [42]:
model.invoke("Add new cat into the list")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'createPets', 'arguments': '{}'}})

### Routing
 THe goal of routing is either invoke the tool (i.e the function) ,that the model conclude that its the right function to be called, by
 AgentActionMessageLog and then return the result of the tool.
 OR to get normal respone from the model by AgentFinish (without calling any function)

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [43]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [46]:
model.invoke("How it looks like in paris today")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":48.8566,"longitude":2.3522}'}})

In [47]:
model.invoke("what is Mlops")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query":"MLOps"}'}})

In [48]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful a assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [49]:
chain.invoke({"input": "what is the weather in LA right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":34.0522,"longitude":-118.2437}'}})

In [50]:
# use OpenAIFunctionsAgentOutputParser to invoke the tool with the input from the model 
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [51]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [58]:
result = chain.invoke({"input": "what is the weather in Angers right now"})

In [59]:
result

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 47.4784, 'longitude': -0.5632}, log="\nInvoking: `get_current_temperature` with `{'latitude': 47.4784, 'longitude': -0.5632}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":47.4784,"longitude":-0.5632}'}})])

In [54]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [55]:
result.tool

'get_current_temperature'

In [60]:
result.tool_input

{'latitude': 47.4784, 'longitude': -0.5632}

In [61]:
# pass the input into the tool
get_current_temperature(result.tool_input)

'The current temperature is 3.1°C'

Invoke the model with normal chat message

In [65]:
result = chain.invoke({"input": "Salut"})

In [68]:
result

AgentFinish(return_values={'output': "Salut! Comment puis-je vous aider aujourd'hui?"}, log="Salut! Comment puis-je vous aider aujourd'hui?")

In [66]:
type(result)

langchain.schema.agent.AgentFinish

In [67]:
result.return_values

{'output': "Salut! Comment puis-je vous aider aujourd'hui?"}

#### THis function will take the result of the chatModel, then verify which agent has been called. 
#### If it's Agentfinish, then show the respone of the model.
#### If not, then call the tool in result.tool and pass to it the tool_input from the chatModel to get the output

In [81]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [82]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [83]:
result = chain.invoke({"input": "What is the weather in Lyon right now?"})

In [84]:
result

'The current temperature is 7.0°C'

In [73]:
result = chain.invoke({"input": "What is GPU?"})

In [74]:
result

"Page: Graphics processing unit\nSummary: A graphics processing unit (GPU) is a specialized electronic circuit initially designed for digital image processing and to accelerate computer graphics, being present either as a discrete video card or embedded on motherboards, mobile phones, personal computers, workstations, and game consoles. After their initial design, GPUs were found to be useful for non-graphic calculations involving embarrassingly parallel problems due to their parallel structure. Other non-graphical uses include the training of neural networks and cryptocurrency mining.\n\nPage: List of Nvidia graphics processing units\nSummary: This list contains general information about graphics processing units (GPUs) and video cards from Nvidia, based on official specifications. In addition some Nvidia motherboards come with integrated onboard GPUs. Limited/Special/Collectors' Editions or AIB versions are not included.\n\n\n\nPage: WebGPU\nSummary: WebGPU is a JavaScript API provid

In [86]:
chain.invoke({"input": "Goodbye"})

'Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!'